# Recommendation

## Import Dependencies

In [0]:
from collections import Counter
from itertools import combinations
from math import sqrt
import random
from keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow

Using TensorFlow backend.


In [0]:
STUDENT_ID = ""

In [0]:
random.seed(2019)
np.random.seed(2019)
tensorflow.set_random_seed(2019)

## Functions

In [0]:
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [0]:
def build_deepwide_model(len_continuous, deep_vocab_lens, len_wide, embed_size):
    input_list = []
    continuous_input = Input(shape=(len_continuous,), dtype='float32', name='continuous_input')
    input_list.append(continuous_input)

    emb_list = []
    for vocab_size in deep_vocab_lens:
        _input = Input(shape=(1,), dtype='int32')
        input_list.append(_input)
        _emb = Embedding(output_dim=embed_size, input_dim=vocab_size, input_length=1)(_input)
        _emb = Reshape((embed_size,))(_emb)
        emb_list.append(_emb)

    deep_input = Concatenate()(emb_list + [continuous_input])
    dense_1 = Dense(256, activation='relu')(deep_input)
    dense_1_dp = Dropout(0.0)(dense_1)
    dense_2 = Dense(128, activation='relu')(dense_1_dp)
    dense_2_dp = Dropout(0.0)(dense_2)
    dense_3 = Dense(64, activation='relu')(dense_2_dp)
    dense_3_dp = Dropout(0.0)(dense_3)

    wide_input = Input(shape=(len_wide,), dtype='float32')
    input_list.append(wide_input)
    
    fc_input = Concatenate()([dense_3_dp, wide_input])
    model_output = Dense(1)(fc_input)
    model = Model(inputs=input_list,
                  outputs=model_output)
    return model

In [0]:
def get_continuous_features(df, continuous_columns):
    continuous_features = df[continuous_columns].values
    return continuous_features

In [0]:
def get_top_k_p_combinations(df, comb_p, topk, output_freq=False):
    def get_category_combinations(categories_str, comb_p=2):
        categories = categories_str.split(', ')
        return list(combinations(categories, comb_p))
    all_categories_p_combos = df["item_categories"].apply(
        lambda x: get_category_combinations(x, comb_p)).values.tolist()
    all_categories_p_combos = [tuple(t) for item in all_categories_p_combos for t in item]
    tmp = dict(Counter(all_categories_p_combos))
    sorted_categories_combinations = list(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    if output_freq:
        return sorted_categories_combinations[:topk]
    else:
        return [t[0] for t in sorted_categories_combinations[:topk]]

In [0]:
def get_wide_features(df):
    def categories_to_binary_output(categories):
        binary_output = [0 for _ in range(len(selected_categories_to_idx))]
        for category in categories.split(', '):
            if category in selected_categories_to_idx:
                binary_output[selected_categories_to_idx[category]] = 1
            else:
                binary_output[0] = 1
        return binary_output
    def categories_cross_transformation(categories):
        current_category_set = set(categories.split(', '))
        corss_transform_output = [0 for _ in range(len(top_combinations))]
        for k, comb_k in enumerate(top_combinations):
            if len(current_category_set & comb_k) == len(comb_k):
                corss_transform_output[k] = 1
            else:
                corss_transform_output[k] = 0
        return corss_transform_output

    category_binary_features = np.array(df.item_categories.apply(
        lambda x: categories_to_binary_output(x)).values.tolist())
    category_corss_transform_features = np.array(df.item_categories.apply(
        lambda x: categories_cross_transformation(x)).values.tolist())
    return np.concatenate((category_binary_features, category_corss_transform_features), axis=1)

## Hyperparameters

In [0]:
EMBEDDING_SIZE = 100

In [0]:
EPOCHS = 10
LEARNING_RATE = 0.0001
BATCH_SIZE = 128

## Main Function

In [0]:
tr_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/valid.csv")
te_df = pd.read_csv("data/test.csv")

In [0]:
tr_ratings = tr_df.stars.values
val_ratings = val_df.stars.values

In [0]:
user_df = pd.read_json("data/user.json")
item_df = pd.read_json("data/business.json")
user_df = user_df.rename(index=str, columns={t: 'user_' + t for t in user_df.columns if t != 'user_id'})
item_df = item_df.rename(index=str, columns={t: 'item_' + t for t in item_df.columns if t != 'business_id'})

In [0]:
tr_df["index"] = tr_df.index
val_df["index"]  = val_df.index
te_df["index"] = te_df.index
tr_df = pd.merge(pd.merge(tr_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
val_df = pd.merge(pd.merge(val_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
te_df = pd.merge(pd.merge(te_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)

## Feature Engineering

In [0]:
attributes = tr_df["item_attributes"].apply(lambda attributes: attributes.keys() if attributes is not None else ()).values
all_attributes = []
for review_attributes in attributes:
    for attribute in review_attributes:
        all_attributes.append(attribute)
all_attributes, all_attributes_count = np.unique(all_attributes, return_counts=True)
all_attributes = all_attributes[np.flipud(np.argsort(all_attributes_count))]
all_attributes_count = np.flipud(np.sort(all_attributes_count))
print(all_attributes, all_attributes_count)

['RestaurantsPriceRange2' 'BusinessParking' 'BikeParking' 'WiFi'
 'GoodForKids' 'BusinessAcceptsCreditCards' 'RestaurantsTakeOut'
 'OutdoorSeating' 'Alcohol' 'Caters' 'RestaurantsGoodForGroups' 'HasTV'
 'RestaurantsDelivery' 'Ambience' 'NoiseLevel' 'RestaurantsReservations'
 'RestaurantsAttire' 'GoodForMeal' 'RestaurantsTableService'
 'WheelchairAccessible' 'Music' 'BusinessAcceptsBitcoin' 'BestNights'
 'GoodForDancing' 'HappyHour' 'DogsAllowed' 'BYOBCorkage' 'CoatCheck'
 'Smoking' 'ByAppointmentOnly' 'Corkage' 'DriveThru' 'BYOB' 'AgesAllowed'
 'AcceptsInsurance' 'DietaryRestrictions' 'Open24Hours'
 'RestaurantsCounterService' 'HairSpecializesIn'] [94646 94417 91910 87877 87635 87404 86762 86171 85789 85162 84574 83762
 83756 83621 83233 83230 81647 75913 40984 25519 18664 17012 16751 16389
 14233 13518 11845 10414 10226  8890  6559  6398   544   478   335   205
   177   167    56]


In [0]:
def get_attribute_value(attributes, attribute_key, value_map, not_exist_value):
    if attribute_key in attributes:
        for value, parsed_value in value_map:
            if attributes[attribute_key] == value:
                return parsed_value
    return not_exist_value

tr_df["item_attributes_parsed"] = tr_df["item_attributes"].apply(lambda attributes: attributes if attributes is not None else {})
val_df["item_attributes_parsed"] = val_df["item_attributes"].apply(lambda attributes: attributes if attributes is not None else {})
te_df["item_attributes_parsed"] = te_df["item_attributes"].apply(lambda attributes: attributes if attributes is not None else {})

tr_df["item_RestaurantsPriceRange2"] = tr_df["item_attributes_parsed"].apply(lambda attributes: int(attributes["RestaurantsPriceRange2"]) if attributes is not None and "RestaurantsPriceRange2" in attributes else 0)
tr_df["item_BikeParking"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BikeParking", [('True', 1), ('False', 0)], -1))
tr_df["item_GoodForKids"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "GoodForKids", [('True', 1), ('False', 0)], -1))
tr_df["item_BusinessAcceptsCreditCards"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BusinessAcceptsCreditCards", [('True', 1), ('False', 0)], -1))
tr_df["item_RestaurantsTakeOut"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTakeOut", [('True', 1), ('False', 0)], -1))
tr_df["item_OutdoorSeating"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "OutdoorSeating", [('True', 1), ('False', 0)], -1))
tr_df["item_Alcohol"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Alcohol", [("full_bar", 0), ("u'full_bar'", 0), ("beer_and_wine", 1), ("u'beer_and_wine'", 1)], -1))
tr_df["item_Caters"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Caters", [('True', 1), ('False', 0)], -1))
tr_df["item_RestaurantsGoodForGroups"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsGoodForGroups", [('True', 1), ('False', 0)], -1))
tr_df["item_HasTV"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "HasTV", [('True', 1), ('False', 0)], -1))
tr_df["item_RestaurantsDelivery"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsDelivery", [('True', 1), ('False', 0)], -1))
tr_df["item_NoiseLevel"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "NoiseLevel", [("u'quiet'", 0), ("quiet", 0), ("u'average'", 1), ("average", 1), ("u'loud'", 2), ("loud", 2), ("u'very_loud'", 3), ("very_loud", 3)], -1))
tr_df["item_RestaurantsReservations"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsReservations", [('True', 1), ('False', 0)], -1))
tr_df["item_RestaurantsAttire"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsAttire", [("u'casual'", 0), ("casual", 0), ("u'dressy'", 1), ("dressy", 1)], -1))
tr_df["item_RestaurantsTableService"] = tr_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTableService", [('True', 1), ('False', 0)], -1))

val_df["item_RestaurantsPriceRange2"] = val_df["item_attributes_parsed"].apply(lambda attributes: int(attributes["RestaurantsPriceRange2"]) if attributes is not None and "RestaurantsPriceRange2" in attributes else 0)
val_df["item_BikeParking"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BikeParking", [('True', 1), ('False', 0)], -1))
val_df["item_GoodForKids"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "GoodForKids", [('True', 1), ('False', 0)], -1))
val_df["item_BusinessAcceptsCreditCards"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BusinessAcceptsCreditCards", [('True', 1), ('False', 0)], -1))
val_df["item_RestaurantsTakeOut"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTakeOut", [('True', 1), ('False', 0)], -1))
val_df["item_OutdoorSeating"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "OutdoorSeating", [('True', 1), ('False', 0)], -1))
val_df["item_Alcohol"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Alcohol", [("full_bar", 0), ("u'full_bar'", 0), ("beer_and_wine", 1), ("u'beer_and_wine'", 1)], -1))
val_df["item_Caters"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Caters", [('True', 1), ('False', 0)], -1))
val_df["item_RestaurantsGoodForGroups"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsGoodForGroups", [('True', 1), ('False', 0)], -1))
val_df["item_HasTV"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "HasTV", [('True', 1), ('False', 0)], -1))
val_df["item_RestaurantsDelivery"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsDelivery", [('True', 1), ('False', 0)], -1))
val_df["item_NoiseLevel"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "NoiseLevel", [("u'quiet'", 0), ("quiet", 0), ("u'average'", 1), ("average", 1), ("u'loud'", 2), ("loud", 2), ("u'very_loud'", 3), ("very_loud", 3)], -1))
val_df["item_RestaurantsReservations"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsReservations", [('True', 1), ('False', 0)], -1))
val_df["item_RestaurantsAttire"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsAttire", [("u'casual'", 0), ("casual", 0), ("u'dressy'", 1), ("dressy", 1)], -1))
val_df["item_RestaurantsTableService"] = val_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTableService", [('True', 1), ('False', 0)], -1))

te_df["item_RestaurantsPriceRange2"] = te_df["item_attributes_parsed"].apply(lambda attributes: int(attributes["RestaurantsPriceRange2"]) if attributes is not None and "RestaurantsPriceRange2" in attributes else 0)
te_df["item_BikeParking"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BikeParking", [('True', 1), ('False', 0)], -1))
te_df["item_GoodForKids"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "GoodForKids", [('True', 1), ('False', 0)], -1))
te_df["item_BusinessAcceptsCreditCards"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "BusinessAcceptsCreditCards", [('True', 1), ('False', 0)], -1))
te_df["item_RestaurantsTakeOut"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTakeOut", [('True', 1), ('False', 0)], -1))
te_df["item_OutdoorSeating"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "OutdoorSeating", [('True', 1), ('False', 0)], -1))
te_df["item_Alcohol"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Alcohol", [("full_bar", 0), ("u'full_bar'", 0), ("beer_and_wine", 1), ("u'beer_and_wine'", 1)], -1))
te_df["item_Caters"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "Caters", [('True', 1), ('False', 0)], -1))
te_df["item_RestaurantsGoodForGroups"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsGoodForGroups", [('True', 1), ('False', 0)], -1))
te_df["item_HasTV"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "HasTV", [('True', 1), ('False', 0)], -1))
te_df["item_RestaurantsDelivery"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsDelivery", [('True', 1), ('False', 0)], -1))
te_df["item_NoiseLevel"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "NoiseLevel", [("u'quiet'", 0), ("quiet", 0), ("u'average'", 1), ("average", 1), ("u'loud'", 2), ("loud", 2), ("u'very_loud'", 3), ("very_loud", 3)], -1))
te_df["item_RestaurantsReservations"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsReservations", [('True', 1), ('False', 0)], -1))
te_df["item_RestaurantsAttire"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsAttire", [("u'casual'", 0), ("casual", 0), ("u'dressy'", 1), ("dressy", 1)], -1))
te_df["item_RestaurantsTableService"] = te_df["item_attributes_parsed"].apply(lambda attributes: get_attribute_value(attributes, "RestaurantsTableService", [('True', 1), ('False', 0)], -1))

## Continuous Features

In [0]:
print("Prepare continuous features...")
continuous_columns = [
    "user_average_stars",
#     "user_cool",
#     "user_fans",
    "user_review_count",
    "user_useful",
#     "user_funny",
#     "item_is_open",
#     "item_latitude",
#     "item_longitude",
    "item_review_count",
    "item_stars",
    "item_RestaurantsPriceRange2",
#     "item_BikeParking",
    "item_GoodForKids",
#     "item_BusinessAcceptsCreditCards",
#     "item_RestaurantsTakeOut",
#     "item_OutdoorSeating",
#     "item_Alcohol",
#     "item_Caters",
#     "item_RestaurantsGoodForGroups",
#     "item_HasTV",
#     "item_NoiseLevel",
#     "item_RestaurantsDelivery",
#     "item_RestaurantsReservations",
#     "item_RestaurantsAttire",
#     "item_RestaurantsTableService"
]
tr_continuous_features = get_continuous_features(tr_df, continuous_columns)
val_continuous_features = get_continuous_features(val_df, continuous_columns)
te_continuous_features = get_continuous_features(te_df, continuous_columns)
scaler = StandardScaler().fit(tr_continuous_features)
tr_continuous_features = scaler.transform(tr_continuous_features)
val_continuous_features = scaler.transform(val_continuous_features)
te_continuous_features = scaler.transform(te_continuous_features)

Prepare continuous features...


## Deep Features

In [0]:
print("Prepare deep features...")
item_deep_columns = [
    "item_city",
#     "item_postal_code",
    "item_state"
]
item_deep_vocab_lens = []
for col_name in item_deep_columns:
    tmp = item_df[col_name].unique()
    vocab = dict(zip(tmp, range(1, len(tmp) + 1)))
    item_deep_vocab_lens.append(len(vocab) + 1)
    item_df[col_name + "_idx"] = item_df[col_name].apply(lambda x: vocab[x] if x in vocab else 0)
item_deep_idx_columns = [t + "_idx" for t in item_deep_columns]
item_to_deep_features = dict(zip(item_df.business_id.values, item_df[item_deep_idx_columns].values.tolist()))
tr_deep_features = np.array(tr_df.business_id.apply(lambda x: item_to_deep_features[x]).values.tolist())
val_deep_features = np.array(val_df.business_id.apply(lambda x: item_to_deep_features[x]).values.tolist())
te_deep_features = np.array(te_df.business_id.apply(lambda x: item_to_deep_features[x]).values.tolist())

Prepare deep features...


## Wide Features

In [0]:
#   Prepare binary encoding for each selected categories
all_categories = [category for category_list in item_df.item_categories.values for category in category_list.split(", ")]
category_sorted = sorted(Counter(all_categories).items(), key=lambda x: x[1], reverse=True)
selected_categories = [t[0] for t in category_sorted[:500]]
selected_categories_to_idx = dict(zip(selected_categories, range(1, len(selected_categories) + 1)))
selected_categories_to_idx['unk'] = 0
idx_to_selected_categories = {val: key for key, val in selected_categories_to_idx.items()}

In [0]:
#   Prepare Cross transformation for each categories
top_combinations = []
top_combinations += get_top_k_p_combinations(tr_df, 2, 20, output_freq=False)
top_combinations += get_top_k_p_combinations(tr_df, 3, 15, output_freq=False)
top_combinations += get_top_k_p_combinations(tr_df, 4, 15, output_freq=False)
top_combinations = [set(t) for t in top_combinations]

In [0]:
tr_wide_features = get_wide_features(tr_df)
val_wide_features = get_wide_features(val_df)
te_wide_features = get_wide_features(te_df)

## Input

In [0]:
# Build input
tr_features = []
tr_features.append(tr_continuous_features.tolist())
tr_features += [tr_deep_features[:,i].tolist() for i in range(len(tr_deep_features[0]))]
tr_features.append(tr_wide_features.tolist())
val_features = []
val_features.append(val_continuous_features.tolist())
val_features += [val_deep_features[:,i].tolist() for i in range(len(val_deep_features[0]))]
val_features.append(val_wide_features.tolist())
te_features = []
te_features.append(te_continuous_features.tolist())
te_features += [te_deep_features[:,i].tolist() for i in range(len(te_deep_features[0]))]
te_features.append(te_wide_features.tolist())

## Train

In [0]:
# Model training
deepwide_model = build_deepwide_model(
    len(tr_continuous_features[0]),
    item_deep_vocab_lens,  
    len(tr_wide_features[0]), 
    embed_size=EMBEDDING_SIZE)
deepwide_model.compile(
    optimizer='adagrad',
    loss='mse'
)
history = deepwide_model.fit(
    tr_features,
    tr_ratings,
    validation_data=(val_features, val_ratings),
    epochs=EPOCHS,
    verbose=1,
    callbacks=[
        ModelCheckpoint('model.h5'),
        EarlyStopping(restore_best_weights=True)
    ]
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 12s 123us/step - loss: 1.1067 - val_loss: 1.0657
Epoch 2/10
100000/100000 [==============================] - 10s 98us/step - loss: 1.0610 - val_loss: 1.0641
Epoch 3/10
100000/100000 [==============================] - 10s 98us/step - loss: 1.0551 - val_loss: 1.0613
Epoch 4/10
100000/100000 [==============================] - 10s 98us/step - loss: 1.0520 - val_loss: 1.0605
Epoch 5/10
100000/100000 [==============================] - 10s 101us/step - loss: 1.0500 - val_loss: 1.0597
Epoch 6/10
100000/100000 [==============================] - 11s 106us/step - loss: 1.0479 - val_loss: 1.0578
Epoch 7/10
100000/100000 [==============================] - 10s 104us/step - loss: 1.0466 - val_loss: 1.0567
Epoch 8/10
100000/100000 [==============================] - 10s 98us/

## Predict

In [0]:
# Make Prediction
y_pred = deepwide_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = deepwide_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))
y_pred = deepwide_model.predict(te_features)
res_df = pd.DataFrame()
res_df['pred'] = y_pred[:, 0]
res_df.to_csv("{}.csv".format(STUDENT_ID), index=False)
print("Writing test predictions to file done.")

TRAIN RMSE:  1.0205543193657187
VALID RMSE:  1.0279644654628406
Writing test predictions to file done.
